In [18]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
import math
import pandas as pd
import random
import wandb

from torch import nn, optim
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR, SequentialLR
from torch.utils.data import DataLoader, Dataset, ConcatDataset

from datasets import load_dataset
from transformers import AutoTokenizer
from staticvectors import StaticVectors
from datetime import datetime
from tqdm import tqdm

from models.LanguageTransformer import LanguageTransformer
from data.AutoregressiveLanguage import AutoregressiveLanguageDataset

In [19]:
# dynamically select device
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [86]:
# load trained model
model_path = "./checkpoints/autoregressive-language-model/alm-2025_12_21_15_12_epoch9_end.pth"
chkpt = torch.load(model_path, weights_only=False, map_location=torch.device(device))

# get model configuration
model_config = chkpt['model_config']

# get vocabulary
vocab = chkpt['vocab']
print(chkpt['loss'])

FileNotFoundError: [Errno 2] No such file or directory: './checkpoints/autoregressive-language-model/alm-2025_12_21_15_12_epoch9_end.pth'

In [66]:
# load language model
model = LanguageTransformer(
    vocab_size=len(vocab),
    embed_dim=model_config['emb_dim'],
    num_layers=model_config['num_layers'],
    num_heads=model_config['num_heads']
)

model.load_state_dict(chkpt['model_state_dict'])
model.to(device)

LanguageTransformer(
  (token_embedding): Embedding(57707, 256)
  (pos_enc): PositionalEncoding()
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-23): 24 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (classifier): Linear(in_features=256, out_features=57707, bias=True)
)

In [70]:
generation_config = {
    'max_len': 100,
    'temperature': 0.75,
    'top_p': 0.95
}

In [83]:
input_text = "tell me something i don't know."

In [85]:
import re

model.eval()

# prepare input
generated_text = torch.tensor([vocab.word2idx['<s>']] + vocab.text2idx(input_text), device=device).unsqueeze(0)

# autoregressive generation
for _ in range(generation_config['max_len']):
    with torch.no_grad():
        out = model(generated_text)[:, -1, :]

        # apply temperature scaling
        out = out / generation_config['temperature']

        # get probabilities
        probs = torch.nn.functional.softmax(out, dim=1)

        # nucleus/top-p filtering
        sorted_probs, sorted_indices = torch.sort(probs, descending=True, dim=1)
        cumulative_probs = torch.cumsum(sorted_probs, dim=1)
        
        # nucleus coverage
        nucleus = cumulative_probs < generation_config['top_p']
        nucleus_size = torch.sum(nucleus).item()
        nucleus[0,nucleus_size] = True

        # apply coverage
        nucleus_probs = torch.zeros_like(probs)
        nucleus_probs = torch.where(nucleus, sorted_probs, sorted_probs)
        nucleus_probs /= torch.sum(nucleus_probs).item()

        # append generated token
        next_token_id = torch.multinomial(nucleus_probs, 1).item()
        next_token = sorted_indices[0][next_token_id].reshape(1, 1)

        generated_text = torch.cat([generated_text, next_token], dim=1)
        if next_token.item() == vocab.word2idx['</s>']:
            break

# decode generated tokens
generated_string = vocab.idx2text(generated_text.squeeze().tolist()[1:-1])
generated_string = re.sub(r'\s+([.,!?])', r'\1', generated_string)
print(generated_string)

tell me something i do n't know. i suppose i 'm a japanese indian, but i 'm a <?> not a german chinese.  i do n't know if i 'm part of a chinese, but since i 'm a japanese, i just do n't think it should be a chinese.  i, once a few people have come to china, have <?> kong.  i 'm very much a chinese - speaking chinese.  but i 'm so impressed. i have been a chinese - korea for a long time, because i have no
